In [ ]:
import numpy as np
import rasterio as rio
import geopandas as gpd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from pyproj import CRS, Transformer
from shapely.geometry import Polygon
from cartopy.feature import ShapelyFeature

# Function to stretch the image values based on percentiles
def percentile_stretch(img, pmin=0., pmax=100.):
    '''
    Stretch image values to the percentile range defined by pmin and pmax.
    '''
    if not 0 <= pmin < pmax <= 100:
        raise ValueError('0 <= pmin < pmax <= 100')
    if not img.ndim == 2:
        raise ValueError('Image can only have two dimensions (row, column)')

    minval = np.percentile(img, pmin)
    maxval = np.percentile(img, pmax)

    stretched = (img - minval) / (maxval - minval)
    stretched[img < minval] = 0
    stretched[img > maxval] = 1

    return stretched

# Function to display the image with optional stretch parameters for each band
def img_display(img, ax, bands, stretch_args=None, **imshow_args):
    '''
    Display the image with optional stretch parameters for each band.
    '''
    dispimg = img.copy().astype(np.float32)

    for b in range(img.shape[0]):
        if stretch_args is None:
            dispimg[b] = percentile_stretch(img[b])
        else:
            dispimg[b] = percentile_stretch(img[b], **stretch_args)

    dispimg = dispimg.transpose([1, 2, 0])
    handle = ax.imshow(dispimg[:, :, bands], **imshow_args)

    return handle, ax

In [ ]:
dataset = rio.open('data_files/NI_Mosaic.tif')

print('{} opened in {} mode'.format(dataset.name,dataset.mode))
print('image has {} band(s)'.format(dataset.count))
print('image size (width, height): {} x {}'.format(dataset.width, dataset.height))
print('band 1 dataype is {}'.format(dataset.dtypes[0])) # note that the band name (Band 1) differs from the list index [0]

In [ ]:
dataset = rio.open('data_files/NI_Mosaic.tif')

print('{} opened in {} mode'.format(dataset.name,dataset.mode))
print('image has {} band(s)'.format(dataset.count))
print('image size (width, height): {} x {}'.format(dataset.width, dataset.height))
print('band 1 dataype is {}'.format(dataset.dtypes[0])) # note that the band name (Band 1) differs from the list index [0]

In [ ]:
print(dataset.bounds)

In [ ]:
print(dataset.crs)

In [ ]:
# Open the satellite image and check its CRS
with rio.open('data_files/NI_Mosaic.tif') as dataset:
    img = dataset.read()  # Read image
    xmin, ymin, xmax, ymax = dataset.bounds  # Get image bounds
    crs = dataset.crs  # Get the CRS of the image
    print(f"Image CRS: {crs}")
    print(f"Image Bounds: xmin: {xmin}, ymin: {ymin}, xmax: {xmax}, ymax: {ymax}")

# Define the target CRS (geographic coordinates, PlateCarree)
target_crs = CRS.from_epsg(4326)  # WGS 84 (latitude/longitude)

# If the current CRS is different, reproject the bounding box coordinates
if crs != target_crs:
    transformer = Transformer.from_crs(crs, target_crs, always_xy=True)
    xmin, ymin = transformer.transform(xmin, ymin)
    xmax, ymax = transformer.transform(xmax, ymax)

# Check the transformed coordinates
print(f"Reprojected Bounds: xmin: {xmin}, ymin: {ymin}, xmax: {xmax}, ymax: {ymax}")

In [ ]:
# Load the shapefiles for county boundaries, towns, and the NI outline
counties = gpd.read_file('data_files/counties.shp')
towns = gpd.read_file('data_files/towns.shp')
boundaries = gpd.read_file('data_files/NI_outline.shp')

In [ ]:
ni_utm = ccrs.UTM(29) # note that this matches with the CRS of our image
fig, ax = plt.subplots(1, 1, figsize=(10, 10), subplot_kw=dict(projection=ni_utm))

In [ ]:
ni_utm = ccrs.UTM(29) # note that this matches with the CRS of our image
fig, ax = plt.subplots(1, 1, figsize=(10, 10), subplot_kw=dict(projection=ni_utm))

In [ ]:
ax.imshow(img[3], cmap='gray', vmin=200, vmax=5000) # display band 4 as a grayscale image, stretched between 200 and 5000
ax.set_extent([xmin, xmax, ymin, ymax], crs=ni_utm) # set the extent to the image boundary

fig # show the figure

In [ ]:
ax.imshow(img[3], cmap='gray', vmin=200, vmax=5000, transform=ni_utm, extent=[xmin, xmax, ymin, ymax])
ax.set_extent([xmin, xmax, ymin, ymax], crs=ni_utm) # set the extent to the image boundary

fig

In [ ]:
def img_display(image, ax, bands, transform, extent):
    '''
    This is where you should write a docstring.
    '''
    # first, we transpose the image to re-order the indices
    dispimg = image.transpose([1, 2, 0])
    
    # next, we have to scale the image.
    dispimg = dispimg / dispimg.max()
    
    # finally, we display the image
    handle = ax.imshow(dispimg[:, :, bands], transform=transform, extent=extent)
    
    return handle, ax

In [ ]:
h, ax = img_display(img, ax, [2, 1, 0], ni_utm, [xmin, xmax, ymin, ymax])
fig # just to save you from scrolling back up to see

In [ ]:
def percentile_stretch(image, pmin=0., pmax=100.):
    '''
    This is where you should write a docstring.
    '''
    # here, we make sure that pmin < pmax, and that they are between 0, 100
    if not 0 <= pmin < pmax <= 100:
        raise ValueError('0 <= pmin < pmax <= 100')
    # here, we make sure that the image is only 2-dimensional
    if not image.ndim == 2:
        raise ValueError('Image can only have two dimensions (row, column)')
    
    minval = np.percentile(image, pmin)
    maxval = np.percentile(image, pmax)
    
    stretched = (image - minval) / (maxval - minval) # stretch the image to 0, 1
    stretched[image < minval] = 0 # set anything less than minval to the new minimum, 0.
    stretched[image > maxval] = 1 # set anything greater than maxval to the new maximum, 1.
    
    return stretched

In [ ]:
def img_display(image, ax, bands, transform, extent, pmin=0, pmax=100):
    '''
    This is where you should write a docstring.
    '''
    dispimg = image.copy().astype(np.float32) # make a copy of the original image,
    # but be sure to cast it as a floating-point image, rather than an integer

    for b in range(image.shape[0]): # loop over each band, stretching using percentile_stretch()
        dispimg[b] = percentile_stretch(image[b], pmin=pmin, pmax=pmax)

    # next, we transpose the image to re-order the indices
    dispimg = dispimg.transpose([1, 2, 0])
    
    # finally, we display the image
    handle = ax.imshow(dispimg[:, :, bands], transform=transform, extent=extent)
    
    return handle, ax

In [ ]:
h, ax = img_display(img, ax, [2, 1, 0], ni_utm, [xmin, xmax, ymin, ymax], pmin=0.1, pmax=99.9)
fig # just to save you from scrolling back up to see

In [ ]:
def new_img_display(image, ax, bands, stretch_args=[0, 100], **imshow_args):
    '''
    This is where you should write a docstring.
    '''
    dispimg = image.copy().astype(np.float32) # make a copy of the original image,
    # but be sure to cast it as a floating-point image, rather than an integer

    for b in range(image.shape[0]): # loop over each band, stretching using percentile_stretch()
        dispimg[b] = percentile_stretch(image[b], *stretch_args) # pass the iterable stretch_args, but unpack them when calling percentile_stretch

    # next, we transpose the image to re-order the indices
    dispimg = dispimg.transpose([1, 2, 0])
    
    # finally, we display the image
    handle = ax.imshow(dispimg[:, :, bands], **imshow_args)
    
    return handle, ax

In [ ]:
disp_kwargs = {'extent': [xmin, xmax, ymin, ymax],
               'transform': ni_utm}

stretch = [0.1, 99.9] # a list of percentile values

h, ax = new_img_display(img, ax, [2, 1, 0], stretch_args=stretch, **disp_kwargs)
fig

In [ ]:
# write your code here!
gridlines = ax.gridlines(draw_labels=True, # draw  labels for the grid lines
                         xlocs=[-8, -7.5, -7, -6.5, -6, -5.5], # add longitude lines at 0.5 deg intervals
                         ylocs=[54, 54.5, 55, 55.5]) # add latitude lines at 0.5 deg intervals
gridlines.left_labels = False # turn off the left-side labels
gridlines.bottom_labels = False # turn off the bottom labels


In [ ]:
# Load the shapefiles for county boundaries, towns, and the NI outline
counties = gpd.read_file('data_files/counties.shp')
towns = gpd.read_file('data_files/towns.shp')
boundaries = gpd.read_file('data_files/NI_outline.shp')

# Create the figure and axis objects with Cartopy projection
fig, ax = plt.subplots(figsize=(10, 10), dpi=100, subplot_kw={'projection': ccrs.PlateCarree()})

# Set the extent of the map using the reprojected bounding box values
ax.set_extent([xmin, xmax, ymin, ymax], crs=ccrs.PlateCarree())

# Add the NI outline (shapefile)
boundaries_feature = ShapelyFeature(boundaries.geometry, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=2)
ax.add_feature(boundaries_feature)

# Add county boundaries
counties_feature = ShapelyFeature(counties.geometry, ccrs.PlateCarree(), edgecolor='red', facecolor='none', linewidth=2)
ax.add_feature(counties_feature)

# Add town points (blue squares)
ax.scatter(towns.geometry.x, towns.geometry.y, color='blue', s=100, label='Town', marker='s')